In [1]:
import json
import unicodedata
import re
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import os
from pydantic import BaseModel
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

True

In [8]:
class Metadata(BaseModel):
    chapter: str
    article: str
    title: str
    
    
class Data(BaseModel):
    content: str
    metadata: Metadata
    
    
def normalize_text(text):
    return unicodedata.normalize('NFKD', text)

def remove_redundant_space_newline(text):
    return re.sub(r'\s+', ' ', text).strip()

def preprocess(text: str) -> str:
    text = text.split('\n')
    text = list(filter(None, text))
    text = [subtext.strip() for subtext in text]
    return text

In [9]:
def semantic_split(text) -> str:
    llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', temperature=0.3, max_output_tokens=2000)
    template = """
        Hãy là 1 người phân tích văn bản thông minh và giúp tôi phân chia nội dung 1 đoạn văn bản ra từ phần nhỏ logic ngữ nghĩa "semantic split" (có thể bao gồm các bảng) đã được trích xuất từ file html sang định thân thiện với người đọc (đối với các bảng thì chuyển sang dạng CSV thân thiện với người đọc). Thông tin hữu ích cần trích xuất đó là tiêu đề và nội dung của tiêu đề, không được phép để trống và hạn chế mất mát thông tin từng đoạn.
        Nội dung: {text}
        Định dạng đầu ra là định dạng là 1 list JSON object gồm 2 trường "title" và "content" bắt buộc, không trường được phép để trống.
        """

    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm | JsonOutputParser()

    return chain.invoke({'text': text})


In [22]:
def fetch_and_extract_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        data = []
        chapters_content = soup.find_all('p', id=lambda x: x and x.startswith('Chuong'))

        for index, chapter_content in enumerate(chapters_content, start=1):
            chapter = normalize_text(chapter_content.find('strong').get_text(strip=True))
            articles = [normalize_text(dt.get_text(strip=False)) for dt in soup.find_all('dt')]
            contents = [normalize_text(dd.get_text(strip=False)) for dd in soup.find_all('dd')]
            index_ranges = {
                1: (0, 4),
                2: (4, 20),
                3: (20, 30),
                4: (30, 33),
                5: (33, 37)
            }

            start_idx, end_idx = index_ranges.get(index, (0, 0))
            for article, content in zip(articles[start_idx:end_idx], contents[start_idx:end_idx]):
                print(article)
                contents = semantic_split(content)
                print(contents)
                for content in contents:
                    metadata = dict(chapter=chapter, article=article, title=content['title'])
                    doc = dict(content=content['content'], metadata=metadata)
                    data.append(doc)

        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")

In [23]:
url = "https://undergrad.tdtu.edu.vn/hoc-vu/quy-che-chuc-va-quan-ly-dao-tao-trinh-do-dai-hoc-khoa-ts2021-tro-ve-sau"

# Fetch and extract text from the URL
extracted_data = fetch_and_extract_text(url)
# Save to JSON file
with open('data/json/tdtu_regulations_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(extracted_data, json_file, ensure_ascii=False, indent=2)

Điều 1. Phạm vi và đối tượng áp dụng
[{'title': 'Mục đích và phạm vi điều chỉnh', 'content': 'Quy chế này quy định tổ chức và quản lý đào tạo trình độ đại học hệ chính quy theo hệ thống tín chỉ tại Trường Đại học Tôn Đức Thắng, bao gồm: Chương trình đào tạo, tổ chức đào tạo, đánh giá kết quả học tập, xét và công nhận tốt nghiệp và những quy định khác.'}, {'title': 'Phạm vi áp dụng', 'content': 'Quy chế này áp dụng đối với sinh viên chương trình đào tạo tiêu chuẩn, chương trình đào tạo chất lượng cao, chương trình đào tạo giảng dạy bằng tiếng Anh hệ chính quy trình độ đại học tại Trường Đại học Tôn Đức Thắng (sau đây gọi tắt là Trường) theo hình thức tích lũy tín chỉ.'}]
Điều 2. Chương trình đào tạo
[{'title': 'Cấu trúc chương trình đào tạo', 'content': 'Chương trình đào tạo (CTĐT) được xây dựng theo đơn vị tín chỉ, cấu trúc từ các môn học hoặc học phần (gọi chung là môn học), trong đó phải có đủ các môn học bắt buộc và đáp ứng chuẩn chương trình đào tạo theo quy định 